In [ ]:
%pip install google-generativeai langchain-google-genai


In [3]:
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
import os

from new_secret_key import google_api_key

os.environ["GOOGLE_API_KEY"] = google_api_key

In [6]:
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key= google_api_key)

In [25]:
text = "Give me a short explaination in 6 lines regarding nuclear fusion"

print(gemini_llm.predict(text))

1. Nuclear fusion is a process in which two atomic nuclei combine to form a single heavier nucleus.
2. This process releases a large amount of energy, as the mass of the resulting nucleus is less than the sum of the masses of the original nuclei.
3. The energy released by nuclear fusion is many times greater than that released by nuclear fission, the process used in nuclear power plants.
4. Nuclear fusion is the process that powers the sun and other stars.
5. Scientists are working to develop a way to harness nuclear fusion for energy production on Earth, but this has not yet been achieved.
6. If successful, nuclear fusion could provide a clean and abundant source of energy for the future.


In [14]:
name_prompt = PromptTemplate(input_variables=["cuisine"], template= "Give me one single fancy name for {cuisine} restaurant") 
menu_prompt = PromptTemplate(input_variables=["restaurant_name"], template = "Give me only 3 menu items name for my {restaurant_name}")

name_chain = LLMChain(llm=gemini_llm, prompt= name_prompt, output_key="restaurant_name")
menu_chain = LLMChain(llm=gemini_llm, prompt= menu_prompt, output_key="menu_items")

In [20]:
from langchain.chains import SequentialChain

restaurant_chain = SequentialChain(chains=[name_chain, menu_chain],
                                   input_variables=['cuisine'],
                                   output_variables=['restaurant_name','menu_items'])

restaurant_chain({'cuisine':'french'})

{'cuisine': 'french',
 'restaurant_name': '"Le Jardin de la Gastronomie"',
 'menu_items': '1. **Poème de Filet Mignon:** Tender filet mignon adorned with a delicate wild mushroom sauce and served with a symphony of roasted vegetables.\n\n2. **Valsette de Fruits de Mer:** A delightful medley of fresh seafood, including lobster, shrimp, and scallops, sautéed in a light butter sauce and served with a side of jasmine rice.\n\n3. **Sérénade de Fraises:** A sweet crescendo of fresh strawberries, mascarpone cream, and a hint of lemon zest, served with a crispy almond tuile.'}